# Codigo EEMRIO by Martínez-Hernando


This information is necessary in Python, for activate the funtions 
If they are not installed is necessary to do it, write pip install XXXXX. If pip doesn't work put pip3, or pip3.11 (search on internet)

Activate generical information

In [1]:
import pandas as pd #To real CSV and dataframes
import numpy as np #To work with matrix
from scipy import io, integrate, linalg, signal #Google told me, I don´t know what is it
from scipy.sparse.linalg import cg, eigs #Google told me, I don´t know what is it

To create list

In [ ]:
from typing import List, Tuple, Dict, Callable, Iterable, Union
import itertools
import json
from pathlib import Path


## Importing EXIOBASE 3.3.18 data
We will only import:
- $Z$, the intermediate consumption,
- $Y$, the final demand,
- $F$ and $F_Y$, the environmental extensions, only for air emissions.

In [3]:
exiobase_folder="P:/Universidad/Exiobase/" #Put here your folder where you have download Exiobase with /, no with \

Z (Intermediate products)

In [4]:
Z_raw=pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_by_product_technology.csv", #You should have this name of file, if not change it
                    index_col = [0,1,2,3,4],
                    header = [0,1,2,3])
Z_raw.index.names=["region","sector","code1","code2","units"]
Z_raw.head() #To see the first lines

AU  \
                                                   Cultivation of paddy rice   
                                                                       i01.a   
                                                                      A_PARI   
region sector                  code1 code2  units                              
AU     Paddy rice              p01.a C_PARI tonnes                       0.0   
       Wheat                   p01.b C_WHEA tonnes                       0.0   
       Cereal grains nec       p01.c C_OCER tonnes                       0.0   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                       0.0   
       Oil seeds               p01.e C_OILS tonnes                       0.0   

                                                                         \
                                                   Cultivation of wheat   
                                                                  i01.b   
                                                                 A_WHEA   
region sector                  code1 code2  units                         
AU     Paddy rice              p01.a C_PARI tonnes                  0.0   
       Wheat                   p01.b C_WHEA tonnes                  0.0   
       Cereal grains nec       p01.c C_OCER tonnes                  0.0   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                  0.0   
       Oil seeds               p01.e C_OILS tonnes                  0.0   

                                                                                     \
                                                   Cultivation of cereal grains nec   
                                                                              i01.c   
                                                                             A_OCER   
region sector                  code1 code2  units                                     
AU     Paddy rice              p01.a C_PARI tonnes                          0.00000   
       Wheat                   p01.b C_WHEA tonnes                          0.00000   
       Cereal grains nec       p01.c C_OCER tonnes                      70942.96547   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                          0.00000   
       Oil seeds               p01.e C_OILS tonnes                          0.00000   

                                                                                           \
                                                   Cultivation of vegetables, fruit, nuts   
                                                                                    i01.d   
                                                                                   A_FVEG   
region sector                  code1 code2  units                                           
AU     Paddy rice              p01.a C_PARI tonnes                                0.00000   
       Wheat                   p01.b C_WHEA tonnes                                0.00000   
       Cereal grains nec       p01.c C_OCER tonnes                                0.00000   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                            18266.75262   
       Oil seeds               p01.e C_OILS tonnes                                0.00000   

                                                                             \
                                                   Cultivation of oil seeds   
                                                                      i01.e   
                                                                     A_OILS   
region sector                  code1 code2  units                             
AU     Paddy rice              p01.a C_PARI tonnes                      0.0   
       Wheat                   p01.b C_WHEA tonnes                      0.0   
       Cereal grains nec       p01.c C_OCER tonnes                      0.0   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                      0.0   
       Oi

In [5]:
Z_reindexed=Z_raw.droplevel([2,3,4], axis=0).copy() #To remove useless rows
Z_reindexed.index.names=["region","sector"]
Z_reindexed.columns=Z_reindexed.index
Z_reindexed.head()


region                                 AU                          \
sector                         Paddy rice Wheat Cereal grains nec   
region sector                                                       
AU     Paddy rice                     0.0   0.0           0.00000   
       Wheat                          0.0   0.0           0.00000   
       Cereal grains nec              0.0   0.0       70942.96547   
       Vegetables, fruit, nuts        0.0   0.0           0.00000   
       Oil seeds                      0.0   0.0           0.00000   

region                                                            \
sector                         Vegetables, fruit, nuts Oil seeds   
region sector                                                      
AU     Paddy rice                              0.00000       0.0   
       Wheat                                   0.00000       0.0   
       Cereal grains nec                       0.00000       0.0   
       Vegetables, fruit, nuts             18266.75262       0.0   
       Oil seeds                               0.00000       0.0   

region                                                                    \
sector                         Sugar cane, sugar beet Plant-based fibers   
region sector                                                              
AU     Paddy rice                                 0.0               0.00   
       Wheat                                      0.0               0.00   
       Cereal grains nec                          0.0               0.00   
       Vegetables, fruit, nuts                    0.0               0.00   
       Oil seeds                                  0.0        -1206065.87   

region                                                                 ...  \
sector                         Crops nec        Cattle           Pigs  ...   
region sector                                                          ...   
AU     Paddy rice                    0.0  0.000000e+00       0.000000  ...   
       Wheat                         0.0  6.791574e+05  156083.893300  ...   
       Cereal grains nec             0.0  2.277000e+06  823314.442100  ...   
       Vegetables, fruit, nuts       0.0  1.552051e+03     255.322381  ...   
       Oil seeds                     0.0  0.000000e+00       0.000000  ...   

region                                                    WM  \
sector                         Paper for treatment: landfill   
region sector                                                  
AU     Paddy rice                                        0.0   
       Wheat                                             0.0   
       Cereal grains nec                                 0.0   
       Vegetables, fruit, nuts                           0.0   
       Oil seeds                                         0.0   

region                                                                \
sector                         Plastic waste for treatment: landfill   
region sector                                                          
AU     Paddy rice                                                0.0   
       Wheat                                                     0.0   
       Cereal grains nec                                         0.0   
       Vegetables, fruit, nuts                                   0.0   
       Oil seeds                                                 0.0   

region                                                                              \
sector                         Inert/metal/hazardous waste for treatment: landfill   
region sector                                                                        
AU     Paddy rice                                                             0.0    
       Wheat                                                                  0.0    
       Cereal grains nec                                                      0.0    
       Vegetables, fruit, nuts                                

In [6]:
exio3318_Z=Z_reindexed

F (Extensions, total emissions per activity)

In [125]:
F_raw = pd.read_excel(exiobase_folder + "MR_HIOT_2011_v3_3_18_extensions_together.xlsx", #This data is emissions of intermediate production
                      sheet_name="Emiss_act", #You should have this name of file, if not change it.
                      index_col=[0,1,2,3],
                      header=[0,1,2,3])
F_raw.columns=Z_reindexed.columns

F_raw.head()
exio3318_emissions_F=F_raw

In [126]:
exio3318_emissions_F

region                                                                                                                              AU  \
sector                                                                                                                      Paddy rice
Carbon dioxide, fossil                             tonnes       air   Carbon dioxide, fossil                              3.761413e+04
N2O                                                tonnes       air   Dinitrogen monoxide                                 4.409111e+01
CH4                                                tonnes       air   Methane, fossil                                     2.652729e+04
NOX                                                tonnes       air   Nitrogen oxides                                     2.217441e+02
SOx                                                tonnes       air   Sulfur oxides                                       4.666654e+01
NH3                                                tonnes       air   Ammonia                                             5.242698e+02
NMVOC                                              tonnes       air   NMVOC, non-methane volatile organic compounds       5.373167e+01
CO                                                 tonnes       air   Carbon monoxide, fossil                             1.337744e+02
Pb                                                 tonnes       air   Lead II                                             4.751886e-03
Cd                                                 tonnes       air   Cadmium II                                          3.591020e-04
Hg                                                 tonnes       air   Mercury II                                          2.369422e-04
As                                                 tonnes       air   Arsenic ion                                         3.929728e-04
Cr                                                 tonnes       air   Chromium III                                        2.747395e-03
Cu                                                 tonnes       air   Copper ion                                          2.729743e-02
Ni                                                 tonnes       air   Nickel II                                           2.502747e-02
Se                                                 tonnes       air   Selenium IV                                         6.674613e-04
Zn                                                 tonnes       air   Zinc II                                             2.522927e-02
PCB                                                tonnes       air   Polychlorinated biphenyls                           3.007777e-06
PM10                                               tonnes       air   Particulate Matter, > 2.5 um and < 10um             8.783205e+00
PAH (total of 4 components, sum of EM_AIR.43, 4... tonnes       air   PAH, polycyclic aromatic hydrocarbons               0.000000e+00
PM2.5                                              tonnes       air   Particulate Matter, < 2.5 um                        1.199926e+01
Benzo-[a]-pyrene                                   tonnes       air   Benzo(a)pyrene                                      3.810361e-03
Benzo-[b]-fluoranthene                             tonnes       air   Benzo(b)fluoranthene                                5.194243e-03
Benzo-[k]-fluoranthene                             tonnes       air   Benzo(k)fluoranthene                                2.197754e-03
Indeno-[1,2,3-cd]-pyrene                           tonnes       air   Indeno(1,2,3-cd)pyrene                              7.839437e-04
HCB                                                tonnes       air   Benzene, hexachloro-                                1.102642e-08
PCDD/F (dioxins and furans)                        tonnes I-TEQ air   Dioxins, measured as 2,3,7,8-tetrachlorodibenzo...  6.705266e-09
N                                                  tonnes       water Nitrogen    

In [9]:
X_raw = pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_principal_production.csv", #You should have this name of file, if not change it
                    header = [0,1,2,3,4,5,6,7])

In [10]:
X_reindexed=X_raw.droplevel([2,3,4,5,6,7], axis=1).copy()
X_reindexed.columns=exio3318_Z.columns #To rename the columns
X_reindexed.index=['Total production'] #To rename the index
exio3318_X=X_reindexed

In [11]:
A=exio3318_Z/exio3318_X.squeeze() #To calculate the matrix A
A=A.fillna(0) #To fill the NaN values with 0
n= len(A.index) #To know the number of rows
Ident=np.eye(n) #To create the identity matrix
I_A=Ident-A #To calculate the matrix I-A

In [12]:
print(I_A.shape) #To know the shape of the matrix
I_A.isna().any().any()
print(type(I_A)) #To know the type of the matrix

(7872, 7872)
<class 'pandas.core.frame.DataFrame'>


In [220]:
exio3318_emissions_S=exio3318_emissions_F/exio3318_X.squeeze()
exio3318_emissions_S=exio3318_emissions_S.fillna(0)
exio3318_emissions_S=exio3318_emissions_S.replace([np.inf, -np.inf], 0)
exio3318_emissions_S=exio3318_emissions_S*1000 #To have all in kilograms/cubic meters
exio3318_emissions_S.iloc[-1]=exio3318_emissions_S.iloc[-1]/1000 #Qalys dont have to be changed

# Importation for BW2.5

Libraries for brightway2.5

In [14]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
from bw2io.importers.io import IOImporter
from bw2io.strategies.io import tidy_tables

print('bw2data version: ',bd.__version__)
print('bw2io version: ',bi.__version__)
print('bw2calc version: ',bc.__version__)

bd.projects
bd.projects.set_current('exiobase_db')

if any("forwast" in db for db in bd.databases): #to confirm biosphere is present
    print('forwast is already present in the project.')
else:
    bi.remote.get_projects()
    bi.remote.install_project('forwast','exiobase_db', overwrite_existing=True)

16:16:30+0200 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.
bw2data version:  (4, 0)
bw2io version:  0.9.3
bw2calc version:  2.0.2
forwast is already present in the project.


To confirm biosphere is well installed

In [128]:
methods_installed=len(bd.methods) #This number is not 1, if it is 1; try to run in an other kernel
print(bd.databases)
if methods_installed==1:
    print('You have a problem in brightway installation, only one mtehod have been added. It will be deleted and intalled again')

Databases dictionary with 6 object(s):
	Prueba
	ecoinvent-3.9.1-biosphere
	exiobase_318_hybrid
	exiobase_318_hybrid biosphere
	forwast
	new_biosphere3


New social method (Schools 2025 Peru)

In [221]:
from utils import custom_methods_importer
new_biosphere='new_biosphere3'
if new_biosphere in bd.databases:
    del bd.databases[new_biosphere]
    

lcia_file = Path('social_footprint_method.xlsx')

custom_methods_importer(
    lcia_file='social_footprint_method.xlsx', 
    biosphere_name='ecoinvent-3.9.1-biosphere',
    overwrite=True
)


18:34:45+0200 [info     ] Vacuuming database            


Could not determine dtype for column 5, falling back to string


 ✅ Node QALY per person already exists, passing 

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 0.10 seconds
1 methods
1 cfs
0 unlinked cfs
Wrote matching file to:
C:\Users\marpi\AppData\Local\pylca\Brightway3\exiobase_db.de79a348\output\lcia-matching-errors_custom_lcia.xlsx
Wrote 1 LCIA methods with 1 characterization factors


In [222]:
method = bd.Method(('Social_footprint', 'social category', 'QALY_person'))
method.load()
method.metadata


{
    'description': '',
    'filename': 'social_footprint_method.xlsx',
    'unit': 'QALY/person',
    'abbreviation': 'social_footprintsq.df4f583b4e20e1f1e99b9ed7a0d6088e',
    'num_cfs': 1,
    'geocollections': ['world']
}

# Create database

In [223]:
path_to_intermediate=(Path.cwd()/'example_data')
path_to_intermediate.mkdir(exist_ok=True)

In [224]:
extended_hiot=I_A
extended_S_all=exio3318_emissions_S

for col in extended_hiot.columns:
    extended_hiot[col] = extended_hiot[col].astype(pd.SparseDtype("float", 0))
for col in extended_S_all.columns:
    extended_S_all[col] = extended_S_all[col].astype(pd.SparseDtype("float", 0))


In [225]:
extended_S_all

region                                                                                                                              AU  \
sector                                                                                                                      Paddy rice
Carbon dioxide, fossil                             tonnes       air   Carbon dioxide, fossil                              6.118203e+01
N2O                                                tonnes       air   Dinitrogen monoxide                                 7.171728e-02
CH4                                                tonnes       air   Methane, fossil                                     4.314850e+01
NOX                                                tonnes       air   Nitrogen oxides                                     3.606824e-01
SOx                                                tonnes       air   Sulfur oxides                                       7.590640e-02
NH3                                                tonnes       air   Ammonia                                             8.527616e-01
NMVOC                                              tonnes       air   NMVOC, non-methane volatile organic compounds       8.739834e-02
CO                                                 tonnes       air   Carbon monoxide, fossil                             2.175935e-01
Pb                                                 tonnes       air   Lead II                                             7.729277e-06
Cd                                                 tonnes       air   Cadmium II                                          5.841047e-07
Hg                                                 tonnes       air   Mercury II                                          3.854031e-07
As                                                 tonnes       air   Arsenic ion                                         6.391979e-07
Cr                                                 tonnes       air   Chromium III                                        4.468831e-06
Cu                                                 tonnes       air   Copper ion                                          4.440119e-05
Ni                                                 tonnes       air   Nickel II                                           4.070893e-05
Se                                                 tonnes       air   Selenium IV                                         1.085673e-06
Zn                                                 tonnes       air   Zinc II                                             4.103718e-05
PCB                                                tonnes       air   Polychlorinated biphenyls                           4.892360e-09
PM10                                               tonnes       air   Particulate Matter, > 2.5 um and < 10um             1.428650e-02
PAH (total of 4 components, sum of EM_AIR.43, 4... tonnes       air   PAH, polycyclic aromatic hydrocarbons               0.000000e+00
PM2.5                                              tonnes       air   Particulate Matter, < 2.5 um                        1.951765e-02
Benzo-[a]-pyrene                                   tonnes       air   Benzo(a)pyrene                                      6.197819e-06
Benzo-[b]-fluoranthene                             tonnes       air   Benzo(b)fluoranthene                                8.448801e-06
Benzo-[k]-fluoranthene                             tonnes       air   Benzo(k)fluoranthene                                3.574801e-06
Indeno-[1,2,3-cd]-pyrene                           tonnes       air   Indeno(1,2,3-cd)pyrene                              1.275140e-06
HCB                                                tonnes       air   Benzene, hexachloro-                                1.793525e-11
PCDD/F (dioxins and furans)                        tonnes I-TEQ air   Dioxins, measured as 2,3,7,8-tetrachlorodibenzo...  1.090659e-11
N                                                  tonnes       water Nitrogen    

In [227]:
countries=exio3318_Z.columns.get_level_values(0)
products=exio3318_Z.columns.get_level_values(1)
nick_names=[f"{coun}_{prod}" for coun, prod in zip(countries, products)]
units=np.array(Z_raw.index.get_level_values("units")) #To see the units

sustances=extended_S_all.index.get_level_values(0)
compartments=extended_S_all.index.get_level_values(2)
names_ecoinvent=extended_S_all.index.get_level_values(3)
nick_sustances=[f"{sust}_{comp}" for sust, comp in zip(sustances, compartments)]
print(nick_sustances[-1])
print(names_ecoinvent[-1])

QALY_air
QALY per person


In [228]:
extended_S=extended_S_all.droplevel([1,2,3], axis=0).copy() #To remove unused columns
extended_S.index=nick_sustances

In [230]:
extended_S.iloc[-1].sum()

0.009567017564734744

In [231]:
product_metadata={
    name: {
        "unit": unit,
        "name": name,
        "location": region,
    }
    for name, unit, region in zip(products, units, countries)
}
biosphere_metadata={
    sustance:{'name':sustance,
               'unit':'kilogram', # not standard units
               'compartment':compartment,
    }
    for sustance, compartment in zip(nick_sustances,compartments)
}
biosphere_metadata['Water Withdrawal Blue_air']['unit'] = 'cubic meter' #water has other units
biosphere_metadata['QALY_air']['location'] = 'GLO'
#biosphere_metadata['QALY_social']['unit'] = 'QALY/person'
#biosphere_metadata.popitem()

metadata_dict = {**product_metadata, **biosphere_metadata}

with open(path_to_intermediate/'io_metadata.json', 'w') as fp:
    json.dump(metadata_dict, fp,indent=4)


In [232]:
biosphere_metadata


{
    'Carbon dioxide, fossil_air': {
        'name': 'Carbon dioxide, fossil_air',
        'unit': 'kilogram',
        'compartment': 'air'
    },
    'N2O_air': {'name': 'N2O_air', 'unit': 'kilogram', 'compartment': 'air'},
    'CH4_air': {'name': 'CH4_air', 'unit': 'kilogram', 'compartment': 'air'},
    'NOX _air': {'name': 'NOX _air', 'unit': 'kilogram', 'compartment': 'air'},
    'SOx_air': {'name': 'SOx_air', 'unit': 'kilogram', 'compartment': 'air'},
    'NH3_air': {'name': 'NH3_air', 'unit': 'kilogram', 'compartment': 'air'},
    'NMVOC_air': {'name': 'NMVOC_air', 'unit': 'kilogram', 'compartment': 'air'},
    'CO _air': {'name': 'CO _air', 'unit': 'kilogram', 'compartment': 'air'},
    'Pb_air': {'name': 'Pb_air', 'unit': 'kilogram', 'compartment': 'air'},
    'Cd_air': {'name': 'Cd_air', 'unit': 'kilogram', 'compartment': 'air'},
    'Hg_air': {'name': 'Hg_air', 'unit': 'kilogram', 'compartment': 'air'},
    'As_air': {'name': 'As_air', 'unit': 'kilogram', 'compartment': 'ai

In [233]:
value=[act for act in bd.Database('ecoinvent-3.9.1-biosphere') if act['name']=='QALY per person' and act['categories'] == ('air',)]
value[0]['code']

"QALY per person-('air',)"

Definition of flows

In [234]:
flows_code=[]
for i in range(len(nick_sustances)):
    value=[act for act in bd.Database('ecoinvent-3.9.1-biosphere') if act['name']==names_ecoinvent[i] and act['categories'] == (compartments[i],)]
    codes=value[0]['code']
    flows_code.append(codes)

flows_code_pd=pd.DataFrame(flows_code)
flows_code_pd.index=nick_sustances

In [235]:
flows_code_pd

,0
"Carbon dioxide, fossil_air",349b29d1-3e58-4c66-98b9-9d1a076efd2e
N2O_air,20185046-64bb-4c09-a8e7-e8a9e144ca98
CH4_air,0795345f-c7ae-410c-ad25-1845784c75f5
NOX _air,c1b91234-6f24-417b-8309-46111d09c457
SOx_air,ba5fc0b6-770b-4da1-9b3f-e3b5087f07cd
NH3_air,87883a4e-1e3e-4c9d-90c0-f1bea36f8014
NMVOC_air,d3260d0e-8203-4cbb-a45a-6a13131a5108
CO _air,ba2f3f82-c93a-47a5-822a-37ec97495275
Pb_air,8e123669-94d3-41d8-9480-a79211fe7c43
Cd_air,1c5a7322-9261-4d59-a692-adde6c12de92


In [236]:
biosphere_mapping={}

for name, code in zip(nick_sustances, flows_code):
    biosphere_mapping[name]=code

#biosphere_mapping.popitem()

In [237]:
biosphere_mapping


{
    'Carbon dioxide, fossil_air': '349b29d1-3e58-4c66-98b9-9d1a076efd2e',
    'N2O_air': '20185046-64bb-4c09-a8e7-e8a9e144ca98',
    'CH4_air': '0795345f-c7ae-410c-ad25-1845784c75f5',
    'NOX _air': 'c1b91234-6f24-417b-8309-46111d09c457',
    'SOx_air': 'ba5fc0b6-770b-4da1-9b3f-e3b5087f07cd',
    'NH3_air': '87883a4e-1e3e-4c9d-90c0-f1bea36f8014',
    'NMVOC_air': 'd3260d0e-8203-4cbb-a45a-6a13131a5108',
    'CO _air': 'ba2f3f82-c93a-47a5-822a-37ec97495275',
    'Pb_air': '8e123669-94d3-41d8-9480-a79211fe7c43',
    'Cd_air': '1c5a7322-9261-4d59-a692-adde6c12de92',
    'Hg_air': '71234253-b3a7-4dfe-b166-a484ad15bee7',
    'As_air': 'dc6dbdaa-9f13-43a8-8af5-6603688c6ad0',
    'Cr_air': 'e142b577-e934-4085-9a07-3983d4d92afb',
    'Cu_air': 'ec8144d6-d123-43b1-9c17-a295422a0498',
    'Ni_air': 'a5506f4b-113f-4713-95c3-c819dde6e48b',
    'Se_air': '454c61fd-c52b-4a04-9731-f141bb7b5264',
    'Zn_air': '5ce378a0-b48d-471c-977d-79681521efde',
    'PCB_air': 'c9b97088-efc6-43c1-8d26-f78d6cdbb

In [238]:
print("Columns:", extended_hiot.columns.nlevels) #It must be 2
print("Index:", extended_hiot.index.nlevels) #It must be 2

Columns: 2
Index: 2


In [239]:
print(extended_hiot.shape)
print(extended_S.shape)
print(len(biosphere_mapping))
print(len(biosphere_metadata))

(7872, 7872)
(35, 7872)
35
35


In [240]:
tidy_tables(extended_hiot,extended_S,path_to_intermediate)

In [241]:
name_database="exiobase_318_hybrid" 
if name_database in bd.databases:
    del bd.databases[name_database]

18:42:11+0200 [info     ] Vacuuming database            


In [243]:
exiobase_try1_bw= IOImporter(dirpath=path_to_intermediate,
                         db_name='exiobase_318_hybrid',
                         b3mapping=biosphere_mapping)
exiobase_try1_bw.apply_strategies()
exiobase_try1_bw.write_database()

no extra biosphere flows added


100%|██████████| 7872/7872 [00:03<00:00, 2129.68it/s]


18:42:19+0200 [info     ] Vacuuming database            
18:42:22+0200 [info     ] Starting IO table write       
18:42:23+0200 [info     ] Adding technosphere matrix    
18:44:17+0200 [info     ] Adding biosphere matrix       
18:44:19+0200 [info     ] Finalizing serialization      


In [244]:
test= bd.Database('exiobase_318_hybrid')

In [245]:
locations = {a['location'] for a in test}
names = {a['name'] for a in test}

In [246]:
print(f"{len(locations)} locations, {len(names)} different products")

48 locations, 164 different products


In [247]:
method_chosen=('EF v3.1', 'acidification', 'accumulated exceedance (AE)')
method_social=('Social_footprint', 'social category', 'QALY_person')


In [248]:
act2=next((a for a in test if a['name'] == 'Precious metals' and a['location'] == 'ZA'), None)
print(act2)
print(type(act2))

'Precious metals' (ton, ZA, None)
<class 'bw2data.backends.iotable.proxies.IOTableActivity'>


In [249]:
lca = bc.LCA(
    demand={act2: 1},
    method=method_social
)
lca.lci()
lca.lcia()
lca.score
print(lca.score)



3.807998281747064e-06


In [250]:
lca.to_dataframe().pivot_table(index='row_name',aggfunc='sum',values='amount')

,amount
row_name,
QALY per person,0.000004
